# importing liraries
---

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

Reading csv and setting nulls types
---
---

In [ ]:
data1 = '/kaggle/input/housedata/data.csv'
df = pd.read_csv(data1,na_values=[None,"None",np.nan,"","?"])

Looking for outliers
---
---

In [ ]:
sns.boxplot(data=df["price"])

float price converting to int
---
---
#### we also Removing Rows having Price as zero because cost of a house being zero doesnt make sense

In [ ]:
df.price = df.price.astype(int)
dat = df[df["price"]!=0]
dat

In [ ]:
len(dat)

In [ ]:
dat.info()

Loookup nulls
---
---

In [ ]:
dat.isna().sum()

convert all floats to int because that wont be good for our model
---
---

In [ ]:
cols = dat.select_dtypes(["float64"]).columns
dat[cols] = dat[cols].astype(np.int64)
dat=dat.drop(["statezip"],axis=1)
len(dat)

checking how data looks like by now
---
---

In [ ]:
dat

In [ ]:
dat.date.value_counts()

COnverting date object to Python's date type
---
---

In [ ]:
dat
datz = pd.to_datetime(dat.date).dt.year
datz

## Feature engineering-
### * variable representation
### * variable interaction
### * forming new feature
---
### Important for calculating renovated year , so extracting indexes where yr_renovated is nonzero

In [ ]:
df2 = dat.copy()
df2["rooms"] = df2["bedrooms"]+df2["bathrooms"]
df2["total_sqft"] = df2["sqft_living"] + df2["sqft_lot"]
df2["sqft_above_or_base"] = df2["sqft_above"]+df2['sqft_basement']
df2["yrs_built"] = pd.to_datetime(df2.date).dt.year-df2.yr_built
df2["yrs_renovt"] = 0
df2["mdate"] = pd.to_datetime(df["date"]).dt.year

# Important for calculating renovated year , so extracting indexes where 
li =df2.index[df2['yr_renovated'] >0].tolist()
for ii in li:
    df2.at[ii,"yrs_renovt"]=df2.loc[ii,"mdate"]-df2.loc[ii,"yr_renovated"]
    
df2 = df2.drop(["date","yr_renovated","yr_built","street","country","bathrooms","bedrooms",'sqft_living','sqft_lot','sqft_above','sqft_basement'],axis=1)

df2 = pd.get_dummies(df2)
#df2.isna().sum()
df2

Scaling and splitting
---
---

In [ ]:
X = df2.copy()
target = X.pop("price")
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
Xtr, xtest,ytr,ytest = train_test_split(X, target,test_size=0.33,random_state=42)
Xtr

Applying AdaBoostRegressor , GradientBoostingRegressor
---

In [ ]:

model1 = AdaBoostRegressor()
model1.fit(Xtr,ytr)
model2 = GradientBoostingRegressor()
model2.fit(Xtr,ytr)
print("Model1 accuracy",model1.score(xtest,ytest))
print("Model2 accuracy",model2.score(xtest,ytest))


## Averaging
---

In [ ]:
# initializing all the model objects with default parameters
model_1 = LinearRegression()
model_2 = SVR()
model_3 = RandomForestRegressor()
  
# training all the model on the training dataset
model_1.fit(Xtr,ytr)
model_2.fit(Xtr,ytr)
model_3.fit(Xtr,ytr)
  
# predicting the output on the validation dataset
pred_1 = model_1.predict(xtest)
pred_2 = model_2.predict(xtest)
pred_3 = model_3.predict(xtest)


# final prediction after averaging on the prediction of all 3 models
pred_final = (pred_1+pred_2+pred_3)/3.0
pred_final

score1 = model_1.score(xtest,ytest)
score2 = model_2.score(xtest,ytest)
score3 = model_3.score(xtest,ytest)
avg_score = (score1+score2+score3)/3.0
print(score1,score2,score3,"\nAverage Score=",avg_score)